In [1]:
import os
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import config
import utility.df
import utility.iolib
import utility.plotlib
from utility.feature import Feature

#env = sys.argv[1] if len(sys.argv) > 2 else "dev"

In [2]:
# Setup configuration
cfg = config.ResearchConfig
time_format = cfg.CSV_TIME_FORMAT
date_format = cfg.CSV_DATE_FORMAT
cutoff_date = pd.to_datetime(cfg.CUTOFF_DATE, format=cfg.CSV_DATE_FORMAT)

# Retrieve data
df_subspt, df_lesson, df_incomp, df_crclum, df_pupils = utility.iolib.retrieve_data(cfg)
print("Complete loading data for subscription and lesson history!")

# Filter data
cutoff_date = pd.to_datetime(cfg.CUTOFF_DATE, format=cfg.CSV_DATE_FORMAT)
first_date_impFromData = df_subspt.subscription_start_date.min()

pupils_toBeRemoved = utility.df.filter_subspt_data(
    df_subspt, first_date_impFromData, cutoff_date, remove_annual_subspt=False)
df_lesson1 = df_lesson[~df_lesson['pupilId'].isin(pupils_toBeRemoved)]
df_incomp1 = df_incomp[~df_incomp['pupilId'].isin(pupils_toBeRemoved)]
df_subspt1 = df_subspt[~df_subspt['pupilId'].isin(pupils_toBeRemoved)]

df_subspt1 = utility.df.compute_customer_month(df_subspt1, cfg)

Complete loading data for subscription and lesson history!
By the cutoff date 2018-04-20, there are 1234 active subscriptions.
These subscribers shall be removed from the analysis because we have no evidence to know the lifetime of their subscriptions. 

In the first month of dataset starting from 2014-01-01, there are 154 renewal or new subscriptions.
These subscribers shall be removed from the analysis because we have no evidence to show if they renewed or newly joined. 

In summary, there are 1380/5685 subscribers being removed from the dataset in the analysis. 

Calculate customer month in the subscription table.


100%|██████████| 4305/4305 [00:06<00:00, 692.73it/s] 


In [35]:
df_lesson_daily = df_lesson1.groupby(['pupilId', 'date'])[['score','totalQuestions']].sum()
df_lesson_daily.rename(columns={'score':'score_complete', 'totalQuestions':'question_complete'}, 
                       inplace=True)

df_incomp_combineAttempts = df_incomp1.groupby('incomplete_lesson_log_id')[['pupilId', 'date', 'score','total_questions']].last()
df_incomp_combineAttempts.reset_index(inplace=True)

df_incomp_daily = df_incomp_combineAttempts.groupby(['pupilId', 'date'])[['score','total_questions']].sum()
df_incomp_daily.rename(columns={'score':'score_incomplete', 'total_questions':'question_incomplete'}, 
                       inplace=True)

df = pd.concat([df_lesson_daily, df_incomp_daily], axis=1)
df.fillna(0.0, inplace=True)
df['score'] = df.score_complete + df.score_incomplete
df['num_questions'] = df.question_complete + df.question_incomplete

In [36]:
df.drop(columns=['score_complete', 'score_incomplete', 'question_complete', 'question_incomplete'], inplace=True)

In [37]:
df

score  num_questions
pupilId date                            
655886  2014-04-18   15.0           15.0
        2014-04-19    9.0           10.0
        2014-04-20    6.0           15.0
        2014-04-21   21.0           22.0
        2014-05-18   32.0           45.0
        2014-10-21   56.0           70.0
        2014-10-22   28.0           25.0
        2014-12-23   67.0           76.0
        2015-03-06   25.0           25.0
        2015-05-17   32.0           40.0
658323  2013-12-12   10.0           10.0
        2013-12-27   14.0           15.0
        2014-02-23    5.0            5.0
        2016-03-10  120.0          120.0
        2016-03-11   14.0           14.0
        2016-03-13  158.0          142.0
        2016-03-14    7.0            9.0
        2016-03-16   32.0           34.0
        2016-03-17   22.0           31.0
        2016-03-19   25.0           26.0
        2016-03-21   29.0           31.0
        2016-03-22   54.0           57.0
        2016-03-23   54.0           55.0
        2016-03-25   38.0           40.0
        2016-03-29   72.0           79.0
        2016-04-12   92.0           99.0
        2016-04-13   86.0           92.0
        2016-04-16   52.0           56.0
        2016-04-19   23.0           25.0
        2016-04-20   28.0           34.0
...                   ...            ...
1276697 2018-03-13    3.0           10.0
        2018-03-14    4.0           20.0
1280885 2018-02-13    1.0           10.0
        2018-02-14    2.0            5.0
        2018-02-21    0.0           24.0
        2018-03-04    3.0           20.0
        2018-03-10    0.0           10.0
1281418 2018-02-28    1.0           10.0
        2018-03-17    2.0            8.0
1284479 2018-02-25    0.0           10.0
        2018-02-26    4.0           10.0
        2018-03-10    0.0            4.0
1288697 2018-03-13    3.0           10.0
        2018-03-14    2.0            5.0
        2018-03-20    1.5           10.0
        2018-03-22    1.5            7.0
        2018-03-29    0.0           10.0
        2018-03-30    0.0           10.0
        2018-04-03    0.0            5.0
        2018-04-06    1.0            7.0
        2018-04-09    1.0           26.0
1288973 2018-03-13    2.0           10.0
        2018-03-16    1.0            8.0
        2018-03-19    7.0           20.0
1289172 2018-03-13    3.0           10.0
1289524 2018-03-14    3.0           20.0
        2018-03-15    5.0           10.0
        2018-03-18    2.0           10.0
        2018-03-20    3.0            9.0
        2018-03-21    0.5           49.0

[157058 rows x 2 columns]